## Section 11: Milestone Project - 2

76. Milestone Overview
77. Solution Walkhrough - Card and Deck Classes
78. Solution Walkhrough - Hand and Chip Classes
79. Solution Walkhrough - Functions
80. Solution Walkhrough - Final Gameplay Script

### 76. Milestone Overview

Game: Black Jack   

Here are the requirements:
- You need to create a simple text-based BlackJack game
- The game needs to have one player versus an automated dealer.
- The player can stand or hit.
- The player must be able to pick their betting amount.
- You need to keep track of the player's total money.
- You need to alert the player of wins, losses, or busts, etc...

**You must use OOP and classes in some portion of your game. You can not just use functions in your game. Use classes to help you define the Deck and the Player's hand. There are many right ways to do this, so explore it well!** (observation)   

In [1]:
# imports and definitions
import random
from IPython.display import clear_output

suits = ('♥', '♦', '♠', '♣')
ranks = ('2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A')
values = {'2':2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9, '10':10, 'J':10, 'Q':10, 'K':10, 'A':11}

In [2]:
class Card:
    
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
            
    def __str__(self):
        return '|{r: <2}{s:s}|'.format(r=self.rank,s=self.suit)
    
    def hide(self):
        return '|▓▓▓|'
    
    def get_rank(self):
        return self.rank
    
    def get_suit(self):
        return self.suit
    
print(Card(suits[0], ranks[0]))

|2 ♥|


In [3]:
class Deck:
    
    def __init__(self, suits, ranks):
        self.suits = suits
        self.ranks = ranks
        self.deck = None
        
        self._create_cards()
        self._shuffle()
    
    def __str__(self):
        return 'Deck with %d cards.' % len(self.deck)
        
    def _create_cards(self):
        self.deck = []
        for suit in self.suits:
            for rank in self.ranks:
                card = Card(suit, rank)
                self.deck.append(card)
                
    def _shuffle(self):
        random.shuffle(self.deck)
        
    def get_card(self):
        if len(self.deck):
            return self.deck.pop(0)
        return None

In [4]:
# deck testing

deck = Deck(suits, ranks)
print(deck.get_card()) # remove 1
print(deck.get_card()) # remove 2
print(deck)

|J ♦|
|6 ♣|
Deck with 50 cards.


In [5]:
class Hand:
    
    def __init__(self, values):
        self.deck = []
        self.value = 0
        self.values = values
    
    def __str__(self):
        cards = ' , '.join(['%s' % card for card in self.deck])
        value = self.value
        return '%s, v=%d' % (cards, value)
    
    def hide(self):
        cards = self.deck[0].hide()+' , '
        cards += ' , '.join(['%s' % card for card in self.deck[1:]])
        return '%s, v=%s' % (cards, '?')
    
    def add_card(self,card):
        if card:
            self.deck.append(card)
            self.value = self._get_value()
    
    def get_value(self):
        return self.value
    
    def _get_value(self):
        value = 0
        aces = 0
        c = 0
        for card in self.deck:
            rank = card.get_rank()
            if rank != 'A':
                c += 1
                value += self.values[rank]
            else:
                aces += 1
        for _ in range(aces):
            if value > 21:
                value += 1
            elif value + 11 <= 21 and c == len(self.deck)-1:
                value += 11
            else:
                value += 1
        return value

In [6]:
hand = Hand(values)
hand.add_card(deck.get_card())
hand.add_card(deck.get_card())
hand.add_card(deck.get_card())
print(hand)
print(hand.hide())
print(deck)

|4 ♠| , |8 ♥| , |9 ♥|, v=21
|▓▓▓| , |8 ♥| , |9 ♥|, v=?
Deck with 47 cards.


In [7]:
class Chips:
    
    def __init__(self, value):
        self.total = value
        self.bet = 0
    
    def __str__(self):
        return 'chips=%d, bet=%d' % (self.total, sef.bet)
    
    def set_bet(self, bet):
        self.bet = bet
    
    def get_chips(self):
        return self.total
    
    def win_bet(self):
        self.total += self.bet
    
    def lose_bet(self):
        self.total -= self.bet

In [8]:
class Game:
    
    def __init__(self):
        # game definitions
        self.deck = None
        self.player = None
        self.dealer = None
        self.playing = True
        self.chips = Chips(50)
        # player definitions
        self.wins = 0
        self.loses = 0
    
    # reset hand and deck
    def _reset(self):
        self.deck = Deck(suits, ranks)
        self.player = Hand(values)
        self.dealer = Hand(values)
    
    # input definition
    def _input(self, start, rule, msg):
        player = start
        while not rule(player):
            player = int(input(msg))
        return player
    
    # take a bet
    def _take_bet(self):
        limit = self.chips.get_chips()
        rule = lambda x: 0 < x <= limit
        msg = 'Bet [1, {:d}]: '.format(limit)
        value = self._input(-1, rule, msg)
        # set the bet
        self.chips.set_bet(value)
    
    # hit a card
    def _hit(self, hand):
        card = self.deck.get_card()
        hand.add_card(card)
    
    # hit mutiples cards
    def _take_cards(self, hand, count):
        for _ in range(count):
            self._hit(hand)
    
    # player choice
    def _hit_or_not(self):
        rule = lambda x: 0 <= x <= 1
        msg = 'Stay (0) or Hit (1): '
        value = self._input(-1, rule, msg)
        return value
    
    def _again(self):
        if self.chips.get_chips() <= 0:
            return False
        # again?
        rule = lambda x: 0 <= x <= 1
        msg = 'Again? No (0) or Yes (1): '
        value = self._input(-1, rule, msg)
        return value
    
    # print hands
    def _print(self, init=False):
        if init:
            print('D: ', self.dealer.hide())
        else:
            print('D: ', self.dealer)
        print('P: ', self.player)
    
    def _lose(self):
        print('> Player Lose...')
        self.loses += 1
        self.chips.lose_bet()
    
    def _win(self):
        print('> Player Win!')
        self.wins += 1
        self.chips.win_bet()
    
    def play(self):
        while self.playing:
            #clear_output()
            #- take bet
            self._take_bet()
            #- shuffle the deck
            #   and get cards
            self._reset()
            self._take_cards(self.player, 2)
            self._take_cards(self.dealer, 2)
            
            #- print game
            print('----------------------------\n> Init')
            self._print(True)
            print('> Player Turn')
            while self.player.get_value() < 21:
                #- player turn
                choice = self._hit_or_not()
                if choice:
                    self._hit(self.player)
                    self._print(True)
                else:
                    break
            #- scene 1: player bust
            if self.player.get_value() > 21:
                self._lose()
                self.playing = self._again()
                continue
            print('> Dealer Turn')
            self._print()
            while self.dealer.get_value() < self.player.get_value():
                self._hit(self.dealer)
                self._print()
            #- scene 2: dealer bust
            if self.dealer.get_value() > 21:
                self._win()
                self.playing = self._again()
                continue
            #- scene 3: dealer > player
            elif self.dealer.get_value() > self.player.get_value():
                self._lose()
                self.playing = self._again()
                continue
            #- scene 4: player > dealer
            elif self.dealer.get_value() < self.player.get_value():
                self._win()
                self.playing = self._again()
                continue
            else:
                print('> Tie!')
                self.playing = self._again()
                continue
        print('> Finish the game with {:d} chips.'.format(self.chips.get_chips()))

In [9]:
game = Game()
game.play()

Bet [1, 50]: 10
----------------------------
> Init
D:  |▓▓▓| , |Q ♥|, v=?
P:  |10♥| , |5 ♠|, v=15
> Player Turn
Stay (0) or Hit (1): 1
D:  |▓▓▓| , |Q ♥|, v=?
P:  |10♥| , |5 ♠| , |4 ♦|, v=19
Stay (0) or Hit (1): 0
> Dealer Turn
D:  |2 ♦| , |Q ♥|, v=12
P:  |10♥| , |5 ♠| , |4 ♦|, v=19
D:  |2 ♦| , |Q ♥| , |10♠|, v=22
P:  |10♥| , |5 ♠| , |4 ♦|, v=19
> Player Win!
Again? No (0) or Yes (1): 1
Bet [1, 60]: 30
----------------------------
> Init
D:  |▓▓▓| , |A ♥|, v=?
P:  |Q ♥| , |3 ♠|, v=13
> Player Turn
Stay (0) or Hit (1): 1
D:  |▓▓▓| , |A ♥|, v=?
P:  |Q ♥| , |3 ♠| , |7 ♣|, v=20
Stay (0) or Hit (1): 0
> Dealer Turn
D:  |10♠| , |A ♥|, v=21
P:  |Q ♥| , |3 ♠| , |7 ♣|, v=20
> Player Lose...
Again? No (0) or Yes (1): 0
> Finish the game with 30 chips.
